In [6]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Input,
    Embedding,
    LSTM,
    Dense,
    Attention,
    Concatenate,
    Flatten,
)
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from midiutil import MIDIFile
import pickle


In [8]:
# from google.colab import drive
import os

# drive.mount('/content/drive')
os.chdir("/home/jovyan/workspace/cantus_ai/")

In [9]:
def note_to_midi(note, octave):
    # MIDI note numbers for the notes in octave 0
    note_map = {
        "c": 0,
        "c#": 1,
        "d": 2,
        "d#": 3,
        "e": 4,
        "f": 5,
        "f#": 6,
        "g": 7,
        "g#": 8,
        "a": 9,
        "a#": 10,
        "b": 11,
    }

    # Convert note to lowercase to handle both upper and lower case inputs
    note = note.lower()

    # Calculate the MIDI number
    midi_number = (octave + 1) * 12 + note_map[note]

    return midi_number


def melody_to_midi(melody, rhythm_pattern, velocity_pattern, filename):
    midi = MIDIFile(1)
    midi.addTempo(0, 0, 120)

    for i, note in enumerate(melody):
        pitch_class = note[:-1]
        octave = note[-1]
        pitch = note_to_midi(pitch_class.lower(), int(octave))
        duration = rhythm_pattern[i % len(rhythm_pattern)]
        velocity = velocity_pattern[i % len(velocity_pattern)]
        midi.addNote(0, 0, pitch, i, duration, velocity)  # Add note with duration of 1

    with open(filename, "wb") as output_file:
        midi.writeFile(output_file)


# Function to load the model and dictionaries
def load_resources(model_path, vocab_dict_path, mode_dict_path):
    model = load_model(model_path)
    with open(vocab_dict_path, "rb") as f:
        vocab_dict = pickle.load(f)
    with open(mode_dict_path, "rb") as f:
        mode_dict = pickle.load(f)
    return model, vocab_dict, mode_dict


# Function to predict the element probabilities for a given sequence, mode, and remaining notes
def predict_element_probabilities(
    sequence, mode, notes_left, model, vocab_dict, mode_dict
):
    # Reverse the vocab_dict to get the reverse mapping from indices to elements
    index_to_vocab = {index: element for element, index in vocab_dict.items()}

    # Encode the sequence and mode using the dictionaries
    encoded_sequence = [vocab_dict[element] for element in sequence]
    encoded_mode = np.array([mode_dict[mode]]).reshape(1, 1)

    # Create the position input based on the notes left in the series
    position_input = np.array([[notes_left]])

    # Pad the sequence to match the input shape of the model
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(
        [encoded_sequence], padding="pre"
    )

    # Predict the probabilities
    probabilities = model.predict([padded_sequence, encoded_mode, position_input])[0]

    # Map the probabilities back to the elements
    element_probabilities = {
        index_to_vocab[i]: prob
        for i, prob in enumerate(probabilities)
        if i in index_to_vocab
    }

    return element_probabilities


# Function to extend a sequence to a specified length using temperature-controlled sampling
def extend_sequence(
    initial_sequence, mode, target_length, temperature, model, vocab_dict, mode_dict
):
    sequence = initial_sequence[:]
    while len(sequence) < target_length:
        notes_left = target_length - len(sequence)
        element_probabilities = predict_element_probabilities(
            sequence, mode, notes_left, model, vocab_dict, mode_dict
        )

        # Sort elements by probability
        sorted_elements = sorted(
            element_probabilities.items(), key=lambda item: item[1], reverse=True
        )

        # Determine the number of elements to sample from based on temperature
        num_elements = int(len(sorted_elements) * temperature)
        num_elements = max(1, num_elements)  # Ensure at least one element is sampled

        # Sample the next element based on the adjusted probabilities
        elements, probabilities = zip(*sorted_elements[:num_elements])
        probabilities = np.array(probabilities) / np.sum(
            probabilities
        )  # Normalize probabilities
        next_element = np.random.choice(elements, p=probabilities)

        # Append the next element to the sequence
        sequence.append(next_element)

    return sequence

In [22]:
# Load resources
model, vocab_dict, mode_dict = load_resources(
    "models/v1/mass/melody_prediction_model_with_notes_until_end.h5",
    "models/v1/mass/pitch_encoder.pkl",
    "models/v1/mass/mode_encoder.pkl",
)
# Example usage
initial_sequence = ["A3", "D4"]
mode = "2"
target_length = 16
temperature = 1 # Specify the temperature value
mass_sequence = extend_sequence(
    initial_sequence,
    mode,
    target_length,
    temperature,
    model,
    vocab_dict,
    mode_dict,
)

1/1 [==============================] - 0s 18ms/step


In [23]:
# Load resources
model, vocab_dict, mode_dict = load_resources(
    "models/v1/office/melody_prediction_model_with_notes_until_end.h5",
    "models/v1/office/pitch_encoder.pkl",
    "models/v1/office/mode_encoder.pkl",
)
# Example usage
initial_sequence = ["A3", "D4"]
mode = "2"
target_length = 16
temperature = 1 # Specify the temperature value
office_sequence = extend_sequence(
    initial_sequence,
    mode,
    target_length,
    temperature,
    model,
    vocab_dict,
    mode_dict,
)

1/1 [==============================] - 0s 18ms/step


In [24]:
print(mass_sequence)
print(office_sequence)

['A3', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'A4', 'G4', 'A4', 'G4', 'E4', 'G4', 'F4', 'F4', 'F4']
['A3', 'D4', 'D4', 'D4', 'G4', 'G4', 'G4', 'A4', 'G4', 'G4', 'A4', 'C5', 'A4', 'A4', 'B4', 'A4']


In [ ]:
melody_to_midi(
    similar[0][0], [1, 1, 1], [127, 70, 100, 50], "og1_hypodorian_antiphon.mid"
)